This code was used to load sufficient data involving the streets both perpendicular and parallel to Halsted St. Then, two major intersections from all the covered intersections were chosen to train the single agent PPO model. This code segment was also used in the initial stage of our project to perform data analysis.

In [1]:
import requests
import pandas as pd
import folium
import geopandas as gpd
from shapely.geometry import Point
import plotly.express as px
import sqlite3
import logging
from decimal import Decimal
import numpy as np
from datetime import datetime, time, timedelta

url = "https://data.cityofchicago.org/resource/sxs8-h27x.json"

#Initialize query parameters
params = {
        "$limit": 500000,
        "$offset": 0,
        "$where": "caseless_one_of(street, 'Halsted') OR caseless_one_of(from_street, 'Halsted')" +
        "OR caseless_one_of(to_street, 'Halsted')",
        "$order": "time DESC"
    }

 
response = requests.get(url, params=params)

if response.status_code != 200:
    print("Failed to retrieve data.")
    exit()
print("Data loaded")


df = pd.DataFrame(response.json())

# Check for missing values in latitude/longitude columns
df = df.dropna(subset=["start_latitude", "start_longitude"])

dfHalsted = df[df['street'] == 'Halsted']
dfFromHalsted = df[df['from_street'] == 'Halsted']
dfToHalsted = df[df['to_street'] == 'Halsted']

insert_statement = "INSERT INTO chicago_traffic_dataset_table (time, segment_id, speed, message_count, street, from_street, to_street, hour, day_of_week, start_latitude, start_longitude, end_latitude, end_longitude, time_indices) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"
 
create_table_query = """
CREATE TABLE IF NOT EXISTS chicago_traffic_dataset_table (
    time TEXT,
    segment_id TEXT,
    speed REAL,
    message_count TEXT,
    street TEXT,
    from_street TEXT,
    to_street TEXT,
    hour INTEGER,
    day_of_week INTEGER,
    start_latitude REAL,
    start_longitude REAL,
    end_latitude REAL,
    end_longitude REAL,
    time_indices INTEGER
);
"""
         

Data loaded


In [3]:
flowsegment_df = pd.read_csv('ImportantFlowSegments.csv')

def store_to_sqlite(df, db_name, table_name): #store extracted Chicago traffic data to the database
    if df.empty:
        logging.warning("DataFrame is empty; no data to store.")
        return

    try:
        with sqlite3.connect(db_name) as conn:
            print("Storing in database...")
            print(df.shape[0])
            cursor = conn.cursor()
            cursor.execute(create_table_query)
            conn.commit()
            
            for idx, row in df.iterrows():
                cursor.execute(insert_statement, (row['time'], row['segment_id'], row['speed'], row['message_count'], row['street'], row['from_street'], row['to_street'], row['hour'], row['day_of_week'], row['start_latitude'], row['start_longitude'], row['end_latitude'], row['end_longitude'], row['time_indices']))
            conn.commit()

            logging.info(f"Data successfully stored in {db_name} under {table_name}.")
            print(cursor.execute("SELECT COUNT(*) FROM chicago_traffic_dataset_table").fetchone()[0])
    except sqlite3.Error as e:
        logging.error(f"Database error: {str(e)}")

In [4]:
def clean_data(df):
    """Clean the DataFrame by handling missing values."""
    # Check for missing values
    logging.info("Checking for missing values...")
    missing_values = df.isnull().sum()
    logging.info(f"Missing values: {missing_values[missing_values > 0]}")

    # Forward fill missing values
    df.fillna(method='ffill', inplace=True)

    # Convert any dictionary-type columns to strings
    for col in df.columns:
        if df[col].dtype == 'object':
            df[col] = df[col].apply(lambda x: str(x) if isinstance(x, dict) else x)

    return df

def adjust_negative_values(df):
    new_speeds = []
    for idx, row in df.iterrows():
        if row['speed'] == '-1':
            new_speeds.append('0')
        else:
            new_speeds.append(row['speed'])

    df['speed'] = new_speeds

    return df

In [5]:
# Create a Folium map centered around Chicago
chicago_map = folium.Map(location=[41.8781, -87.6298], zoom_start=12)

data_rows = []
for id, row in dfHalsted.iterrows():
  if int(row['segment_id']) in flowsegment_df['segment_id'].to_list():
      data_rows.append(row)
      folium.Marker(location=[row['start_latitude'], row['start_longitude']],
                  popup = "From " + row['from_street'] + " To " + row['to_street'] + " Start latitude: " 
                  + row['start_latitude'] + " Start longitude: " + row['start_longitude'],
                  icon = folium.Icon(color="green", icon="play")).add_to(chicago_map)


      folium.Marker(location=[row['end_latitude'], row['end_longitude']],
                  popup="From " + row['from_street'] + " To " + row['to_street'] + " End latitude: " 
                  + row['end_latitude'] + " End longitude: " + row['end_longitude'],
                  icon = folium.Icon(color='red', icon='pause')).add_to(chicago_map)


for id, row in dfFromHalsted.iterrows():
  if int(row['segment_id']) in flowsegment_df['segment_id'].to_list():
      data_rows.append(row)
      folium.Marker(location = [row['start_latitude'], row['start_longitude']],
                   popup = "From Halsted to " + row['to_street'] + " Start latitude: " + row['start_latitude']
                    + " Start longitude: " + row['start_longitude'],
                   icon = folium.Icon(color='green', icon='play')).add_to(chicago_map)
      folium.Marker(location=[row['end_latitude'], row['end_longitude']],
                  popup="From Halsted to " + row['to_street'] + " End latitude: " + row['end_latitude'] 
                  + " End longitude: " + row['end_longitude'],
                  icon = folium.Icon(color='red', icon='pause')).add_to(chicago_map)
    
for id, row in dfToHalsted.iterrows():
  if int(row['segment_id']) in flowsegment_df['segment_id'].to_list():
      data_rows.append(row)
      folium.Marker(location = [row['start_latitude'], row['start_longitude']],
                   popup = "From " + row["from_street"] + " to Halsted" + " Start latitude: " + row['start_latitude'] 
                   + " Start longitude: " + row['start_longitude'],
                   icon = folium.Icon(color='green', icon='play')).add_to(chicago_map)
      folium.Marker(location=[row['end_latitude'], row['end_longitude']],
                  popup="From " + row["from_street"] + " to Halsted" + " End latitude: " + row['end_latitude'] 
                  + " End longitude: " + row['end_longitude'],
                  icon = folium.Icon(color='red', icon='pause')).add_to(chicago_map)

In [6]:
# Convert results to DataFrame, clean data, and store in the database
adjacent_df = clean_data(pd.DataFrame(data_rows))

# Convert any dictionary-type columns to strings
for col in adjacent_df.columns:
        if adjacent_df[col].dtype == 'object':
            adjacent_df[col] = adjacent_df[col].apply(lambda x: str(x) if isinstance(x, dict) else x)


            #Store cleaned data to sql database

# Save the map with markers
chicago_map.save('chicago_folium_map.html')

filtered_rows = []
times = []
print(flowsegment_df['segment_id'].to_list())

for idx, row in adjacent_df.iterrows():
    if int(row['segment_id']) in flowsegment_df['segment_id'].to_list():
       time_object = datetime.strptime(row['time'], "%Y-%m-%dT%H:%M:%S.%f")
       hour_minute = time(time_object.hour, time_object.minute)
       times.append(hour_minute)
       filtered_rows.append(row)

start_time = min(times) #set minimum time
new_df = pd.DataFrame(filtered_rows)

C:\Users\t_om\AppData\Local\Temp\ipykernel_15520\1497556278.py:9: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)


[1239, 1203, 1204, 1240, 409, 454, 455, 448, 1238, 1202, 273, 259, 274, 258, 1237, 1201]


In [7]:
adjusted_df = adjust_negative_values(new_df)
index = 0
time_indices = []
for idx, row in adjusted_df.iterrows():
    count = 1
    start = timedelta(hours=start_time.hour, minutes=start_time.minute)
    while (timedelta(hours=times[index].hour, minutes=times[index].minute) - start) >= timedelta(minutes=9):
        start+=timedelta(minutes = 10)
        count+=1
    time_indices.append(count)
    index+=1

adjusted_df['time_indices'] = time_indices

store_to_sqlite(adjusted_df,'traffic_interval_data.db','chicago_traffic_dataset_table')

Storing in database...
62496
62496
